### KnowEnG-Research/Data_Cleanup_Pipeline test all methods notebook (for script run on cloud)
    Use this notebook to test the KnowEnG... /Data_Cleanup_Pipeline Locally:


* Create a directory to hold all this code with the test tools repository cloned into it.
    * call it homely_directory
    * git clone https://github.com/dlanier/keg_test_tools.git


* Clone the Data_Cleanup_Pipeline from github. 
    * git clone https://github.com/KnowEnG/Data_Cleanup_Pipeline.git
    * change to Data_Cleanup_Pipeline/test and run >make env_setup
    * change back to the homely_directory


* Clone the Samples_Clustering_Pipeline from github. 
    * git clone https://github.com/KnowEnG/Data_Cleanup_Pipeline.git
    * change to Samples_Clustering_Pipeline/test and run >make env_setup
    * change back to the homely_directory


* Clone the Clustering_Eval_Pipeline from github. 
    * git clone https://github.com/KnowEnG/Data_Cleanup_Pipeline.git
    * change to Clustering_Eval_Pipeline/test and run >make env_setup
    
    
* Run this jupyter notebook

In [1]:
SHOW_COMMAND_LINE_OUTPUT = True

import warnings
warnings.filterwarnings('ignore')

import os
import sys
import time

import pandas as pd
import numpy as np

dcp_src = '../../Data_Cleanup_Pipeline/src/'
sys.path.insert(1, dcp_src)
import data_cleanup_toolbox as dc

scp_src = '../../Samples_Clustering_Pipeline/src/'
sys.path.insert(1, scp_src)
import sample_clustering_toolbox as sc

cep_src = '../../Cluster_Evaluation_Pipeline/src'
sys.path.insert(1, cep_src)
import clustering_eval_toolbox as ce

sys.path.insert(1, '../../keg_test_tools/src')
# import dcp_test
import samples_clustering_test_module as sc_test

import knpackage.toolbox as kn

# If not running inside a docker container of the KnowEnG variety: Create this Directory
output_data_dir = '../../test/run_dir'
yaml_dir = '../../Data_Cleanup_Pipeline/data/run_files'
yaml_file = 'TEMPLATE_data_cleanup.yml'

In [2]:
def cleanup_test_SC(run_parameters):
    """ sc_test_list_df """
    results_directory_base = run_parameters['results_directory']
    
    MAX_COUNT = 3
    sc_test_list_df = sc_test.get_SC_phenotype_viability_df(
        run_parameters['spreadsheet_data_dir'], run_parameters['pheno_data_dir'])
    sc_test_list_df['STATUS'] = 0
    sc_test_list_df['message'] = 0
    count = 1
    for row_n in list(sc_test_list_df.index):
        if sc_test_list_df['samples_intersect'].loc[row_n] > 0 and count <= MAX_COUNT:
            spreadsheet_name = sc_test_list_df['spreadsheet'].loc[row_n]
            run_parameters['spreadsheet_name_full_path'] = os.path.join(
                run_parameters['spreadsheet_data_dir'], spreadsheet_name)

            phenotype_name = sc_test_list_df['phenotype'].loc[row_n]
            run_parameters['phenotype_name_full_path'] = os.path.join(
                run_parameters['pheno_data_dir'], phenotype_name)
            
            dir_extra = '_' + spreadsheet_name + '_' + phenotype_name
            run_parameters['results_directory'] = kn.create_dir(results_directory_base, dir_extra)

            validation_flag, message = dc.run_samples_clustering_pipeline(run_parameters)
            sc_test_list_df['STATUS'].loc[row_n] = validation_flag
            
            log_filename = os.path.join(run_parameters["results_directory"], "log_samples_clustering_pipeline.yml")
            sc_test_list_df['message'].loc[row_n] = log_filename
            dc.generate_logging(validation_flag, message, log_filename)
            
            if validation_flag:
                print('Success! = ', validation_flag)
                print(run_parameters['spreadsheet_name_full_path'], '\n\t', run_parameters['phenotype_name_full_path'])
#                 print(message)
            else:
                print('Fail ! = ', validation_flag)
                print(run_parameters['spreadsheet_name_full_path'], '\n\t', run_parameters['phenotype_name_full_path'])
                print(message)
            
        count += 1
    
    return sc_test_list_df
    

In [2]:
run_parameters = kn.get_run_parameters(yaml_dir, yaml_file)

#                   Data_Cleanup_Pipeline   Samples Clustering
data_root = '/Users/lanier4/pipeline_spreadsheets/aws_v3'

sc_phenotypes_path = os.path.join(data_root, 'SC_P_spreadsheets')
run_parameters['pheno_data_dir'] = sc_phenotypes_path
run_parameters['phenotype_name_full_path'] = 0
#os.path.join(run_parameters['pheno_data_dir'], 'Hsap.ccle.P.cell_line_info.mixed.c.df')

sc_spreadsheets_path = os.path.join(data_root, 'SC_G_spreadsheets')
run_parameters['spreadsheet_data_dir'] = sc_spreadsheets_path
run_parameters['spreadsheet_name_full_path'] = 0
#os.path.join(run_parameters['spreadsheet_data_dir'], 'Hsap.ccle.G.gene_mut.binary.a.df')

run_parameters['pipeline_type'] = 'samples_clustering_pipeline'

run_parameters['run_directory'] = dcp_src
run_parameters['results_directory'] = '../test/tmp'

run_parameters['SHOW_COMMAND_LINE_OUTPUT'] = True
if SHOW_COMMAND_LINE_OUTPUT:
    for p in list(run_parameters.keys()):
        print(p, ':\t\t', run_parameters[p])

# test_result_df = dcp_test.test_data_cleanup(run_parameters)

SHOW_COMMAND_LINE_OUTPUT :		 True
results_directory :		 ../test/tmp
phenotype_name_full_path :		 0
taxonid :		 9606
spreadsheet_name_full_path :		 0
pheno_data_dir :		 /Users/lanier4/pipeline_spreadsheets/aws_v3/SC_P_spreadsheets
pipeline_type :		 samples_clustering_pipeline
run_directory :		 ../../Data_Cleanup_Pipeline/src/
redis_credential :		 {'password': 'KnowEnG', 'host': 'knowredis.knowhub.org', 'port': 6379}
source_hint :		 
run_file :		 TEMPLATE_data_cleanup.yml
spreadsheet_data_dir :		 /Users/lanier4/pipeline_spreadsheets/aws_v3/SC_G_spreadsheets


In [5]:
#                                                    Only if on redis local - else mind-numbingly slow - ensembl names
t0 = time.time()
sc_test_list_df = cleanup_test_SC(run_parameters)
tt = time.time() - t0
print('Run time: %0.3f'%tt)

Success! =  True
/Users/lanier4/pipeline_spreadsheets/aws_v3/SC_G_spreadsheets/Hsap.ccle.G.gene_mut.binary.a.df 
	 /Users/lanier4/pipeline_spreadsheets/aws_v3/SC_P_spreadsheets/Hsap.ccle.P.cell_line_info.mixed.c.df
Success! =  True
/Users/lanier4/pipeline_spreadsheets/aws_v3/SC_G_spreadsheets/Hsap.ccle.G.gene_mut.binary.a.df 
	 /Users/lanier4/pipeline_spreadsheets/aws_v3/SC_P_spreadsheets/Hsap.ccle.P.cyto_ec50.pos.c.df
Success! =  True
/Users/lanier4/pipeline_spreadsheets/aws_v3/SC_G_spreadsheets/Hsap.ccle.G.gene_mut.binary.a.df 
	 /Users/lanier4/pipeline_spreadsheets/aws_v3/SC_P_spreadsheets/Hsap.ccle.P.cyto_ic50.pos.c.df
Run time: 310.604


In [6]:
sc_test_list_df

,spreadsheet,phenotype,spreadsheet_genes,spreadsheet_samples,samples_intersect,phenotype_samples,phenotype_keys,STATUS,message
0001,Hsap.ccle.G.gene_mut.binary.a.df,Hsap.ccle.P.cell_line_info.mixed.c.df,1632,905,905,1046,12,True,[]
0002,Hsap.ccle.G.gene_mut.binary.a.df,Hsap.ccle.P.cyto_ec50.pos.c.df,1632,905,451,504,24,True,[]
0003,Hsap.ccle.G.gene_mut.binary.a.df,Hsap.ccle.P.cyto_ic50.pos.c.df,1632,905,451,504,24,True,[]
0004,Hsap.ccle.G.lg2_cn_ratio.real.a.df,Hsap.ccle.P.cell_line_info.mixed.c.df,21307,1043,1016,1046,12,0,0
0005,Hsap.ccle.G.lg2_cn_ratio.real.a.df,Hsap.ccle.P.cyto_ec50.pos.c.df,21307,1043,492,504,24,0,0
0006,Hsap.ccle.G.lg2_cn_ratio.real.a.df,Hsap.ccle.P.cyto_ic50.pos.c.df,21307,1043,492,504,24,0,0
0007,Hsap.ccle.G.qnorm_probe.pos.a.df,Hsap.ccle.P.cell_line_info.mixed.c.df,17038,1037,1036,1046,12,0,0
0008,Hsap.ccle.G.qnorm_probe.pos.a.df,Hsap.ccle.P.cyto_ec50.pos.c.df,17038,1037,491,504,24,0,0
0009,Hsap.ccle.G.qnorm_probe.pos.a.df,Hsap.ccle.P.cyto_ic50.pos.c.df,17038,1037,491,504,24,0,0
0010,Hsap.dream11.G.exome_mut.binary.a.df,Hsap.dream11.P.avg_ic50.pos.c.df,16012,85,85,85,70,0,0


In [3]:
t0 = time.time()
sc_test.cleanup_test_SC(run_parameters)
tt = time.time() - t0
print('Run time: %0.3f'%tt)

count = 1
count = 2
count = 3
count = 4
count = 5
count = 6
count = 7
count = 8
count = 9
count = 10
count = 11
count = 12
count = 13
count = 14
count = 15
count = 16
count = 17
count = 18
count = 19
count = 20
count = 21
count = 22
count = 23
count = 24
count = 25
count = 26
count = 27
count = 28
count = 29
count = 30
count = 31
count = 32
count = 33
Run time: 220.064
